In [3]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from   time import time
from   torchvision import datasets, transforms
from   torch import nn, optim

In [4]:
transform   = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

trainset    = datasets.MNIST('train', download = True, train = True , transform = transform)
valset      = datasets.MNIST('val  ', download = True, train = False, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)
valloader   = torch.utils.data.DataLoader(valset  , batch_size = 64, shuffle = True)

In [5]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [6]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization

In [7]:
model = CNN()

criterion      = nn.NLLLoss()
images, labels = next(iter(trainloader))
images         = images.view(images.shape[0], -1)

logps = model(images)            # log probabilities
loss  = criterion(logps, labels) # calculate the NLL loss

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [16, 1, 5, 5], but got 2-dimensional input of size [64, 784] instead